In [2]:
import torch
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
def get_repeated_Indices(list_of_indices):
    #input: string of indices for the tensors. 
    #output: string of repeated indices. code takes in indices and output only the repeated indices
    #Ex: Suppose tensor A has index ijkp, and tensor B has index klpm.
    #then get_repeated_Indices('ijkp', 'klpm') will return the following string: 'kp'
    myList = list_of_indices
    #convert List to string
    myString =''.join(myList)
    #break the string into indivual list of characters ex.  'abc' ->['a','b', 'c']
    myList = list(myString)
    #get the repeated frequencies of each indices
    my_dict = {i:myList.count(i) for i in myList}
    
    repeatedList = []
    for item in my_dict:
        if my_dict[item] > 1:
            repeatedList.append(item)
    return repeatedList

def  remove_Repeated_indices(List_of_indices):
    #inputs: tensor indices in the form of string
    #output: string of non repeated indicies
    #Ex: remove_Repeated_indices('abc', 'cde')
    #output of the example would be: 'abde'
    
    myList = List_of_indices 
    #turn myList into String: Ex: ['abc','cde'] -> 'abccde'
    myString = ''.join(myList)
    #turn back into lists again: Exp: from 'abccde' -> ['a','b','c','c','d','e']
    myList = list(myString)
    repeated_indices = get_repeated_Indices(List_of_indices)
    #print('the repeated list of indices are:', repeated_indices)
    unique_indices = []
    #now we remove repeated indices from myList
    for item in myList:
        if item not in repeated_indices:
            unique_indices.append(item)
    uniqueString = ''.join(unique_indices)   
    return uniqueString



In [3]:
def einSum_Contraction(tensorList, indxList):  #<----should rename this to einSum_Contraction to replace old code
    #Purpose: this function takes a list of tensors, and list of indices, and indix to contract and uses einstien summation to perform contraction
    #ex: tensorList = [tensor1, tensor2, tensor3]
    #indxList   = [indx1, indx2, indx3]
    myList = []
    uniqueIndices = remove_Repeated_indices(indxList)
    inputIndices = [indxList]
    N = len(indxList)
    #myList = [indx1, ',',indx2,',',indx3,'->', uniqueIndices] 
    for i in range(N - 1):
        myList.append(indxList[i])
        myList.append(',') 
    myList.append(indxList[N-1])
    myList.append('->')
    myList.append(uniqueIndices)
    #convert myList to a string: i.e.  [indx1, ',',indx2,',',indx3,'->', uniqueIndices]  - >'ijk,klm,mjp->ilp'
    myString = ''.join(myList)
    #print('myString = ', myString)
    C = torch.einsum(myString, tensorList)
    return C

In [4]:
def computeLoss(approxTensor, targetTensor):
    cost = torch.norm(approxTensor - targetTensor, 'fro') 
    return(cost)


In [5]:
def padTensor(tensor, pad_axis):
    #this is for the discrete optimization
    #this function takes a tensor and append an extra dimension of ~ zeros along the specified axis (we call the pad axis)
    if pad_axis == -1:
        return tensor #don't pad anything
    tensorShape = list(tensor.shape)
    tensorShape[pad_axis] = 1  #increase the dimension up by 1
    zerosPad = torch.rand(tensorShape) *1e-6  #pad with values approx. equal to zero
    padded_tensor = torch.cat([tensor, zerosPad], pad_axis)
    #print('padded_tensor.shape = ', padded_tensor.shape)
    #print('padded_tensor function output = ', padded_tensor)
    return padded_tensor

In [6]:
def increaseRank(Tensor1, Tensor2, indx1, indx2):
    # The indx 1 and index2 represents the indices for tensor 1 and 2 respectively. 
    #There is only one repeated index in the list (indx1, indx2). The repeated index represents the shared edge between
    #the two tensors. For ex: ijkl, lmno
    alpha = get_repeated_Indices([indx1, indx2])
    #convert alpha to string
    alpha = ''.join(alpha)
    # find the position of the repeated index alpha in indx1 and indx2
    padAxes1 = indx1.index(alpha)
    padAxes2 = indx2.index(alpha)  
    
    paddedTensor1 = padTensor(Tensor1, padAxes1)
    paddedTensor2 = padTensor(Tensor2, padAxes2)
    return  paddedTensor1, paddedTensor2

In [9]:
#*******************target tensor
#Global variables: d1, d2, d3, targetTensor
d1 = 2
d2 = 3
d3 = 4
r1 = 2
r2 = 3
r3 = 2

X = torch.rand(d1, r3, r1)
Y = torch.rand(r1, d2, r2)
Z = torch.rand(r2, r3, d3)
print(X.shape)
print(Y.shape)
print(Z.shape)

indx0 = 'ijk'
indx1 = 'klm'
indx2 = 'mjp'

target_Tensor = einSum_Contraction([X, Y, Z], [indx0, indx1, indx2])
print('shapeTargetTensor=', target_Tensor.shape)



torch.Size([2, 2, 2])
torch.Size([2, 3, 3])
torch.Size([3, 2, 4])
shapeTargetTensor= torch.Size([2, 3, 4])


In [41]:
#objective function
def TensorGenerator(r1,r2, r3):
    X = torch.rand(d1, r3, r1)
    Y = torch.rand(r1, d2, r2)
    Z = torch.rand(r2, r3, d3)   
    indx0 = 'ijk'
    indx1 = 'klm'
    indx2 = 'mjp'   
    tensorList1 = [X, Y, Z]
    indxList1 = [indx0, indx1, indx2]  


In [42]:
def solve_Continuous_withInput_rank(r1, r2, r3):
#this function is the objective input
    
#TensorGenerator function is here********************************
    X = torch.rand(d1, r3, r1)
    Y = torch.rand(r1, d2, r2)
    Z = torch.rand(r2, r3, d3)   
    indx0 = 'ijk'
    indx1 = 'klm'
    indx2 = 'mjp'
    tensorList = [X, Y, Z]
    indxList = [indx0, indx1, indx2] 
#***************solve_continuous part****************************
    len_Tensor = len(tensorList)
    len_Indx   = len(indxList)
    iterNum = 400
    
    for i in range(len_Tensor):
        tensorList[i] = tensorList[i].detach()
        tensorList[i].requires_grad = True

    #defines a SGD optimizer to update the parameters
    #optimizer = optim.SGD(tensorList lr = 0.001, momentum=0.2)
    optimizer = optim.Adam(tensorList, lr=0.009)

    for i in range(iterNum):
        optimizer.zero_grad()    
        tensor_approx = einSum_Contraction(tensorList, indxList)
    #################################################################################
        loss_fn = computeLoss(tensor_approx, target_Tensor)
    #################################################################################
        loss_fn.backward()
        optimizer.step()                # the new A,B,C will be A_k+1,B_k+1, C_k+1 after optimizer.step 
    return tensorList, indxList, loss_fn

In [43]:
#**********************************good copy. Don't touch this
A = X_approx0
B = Y_approx0
C = Z_approx0

indxList = [indx0, indx1, indx2]
TensorList_temp = [A, B, C]
iterNum=500
Lost_star = 5


for i in range (len(TensorList_temp)):
    for j in range(i,len(TensorList_temp)):
        if i==j:
            continue
        print(i,j)
        [TensorList_temp[i],TensorList_temp[j]] =  increaseRank(TensorList_temp[i], TensorList_temp[j],  indxList[i], indxList[j])
        [TensorList_temp, indxList, LostList] = solve_Continuous_withInput_rank(r1, r2, r3)
        

0 1
0 2
1 2
